In [22]:
import pandas as pd 

data = pd.read_csv("updated-nasa_power_data.csv")
data 

,T2M,RH2M,WS2M,ALLSKY_SFC_SW_DWN,T2M_MAX,PRECTOTCORR,T2M_MIN
0,-0.02,91.94,1.06,2.72,4.05,0.31,-2.06
1,0.03,95.00,0.77,3.87,4.28,0.16,-3.32
2,0.45,95.06,1.74,2.38,3.69,0.35,-0.93
3,0.05,95.56,2.59,1.58,2.95,0.43,-1.13
4,-0.04,94.69,2.45,0.78,3.36,0.16,-1.77
...,...,...,...,...,...,...,...
1121,1.55,88.75,1.93,6.46,7.44,0.03,-1.41
1122,2.77,91.19,3.00,5.98,7.92,0.04,-1.87
1123,7.69,95.81,3.24,6.46,11.79,0.13,4.47
1124,8.05,91.69,2.50,3.21,11.55,0.11,4.55


In [23]:
import streamlit as st
import requests
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# API Key for OpenCage (you need to sign up to get your own key)
OPEN_CAGE_API_KEY = 'f2a2c8e0de9147c28956f29a4fbbbc4e'

# Fonction pour obtenir la latitude et la longitude d'une ville
def get_lat_lon(city_name):
    url = f"https://api.opencagedata.com/geocode/v1/json?q={city_name}&key={OPEN_CAGE_API_KEY}"
    response = requests.get(url)
    data = response.json()
    if data['results']:
        return data['results'][0]['geometry']['lat'], data['results'][0]['geometry']['lng']
    else:
        st.error("Ville non trouvée. Veuillez vérifier le nom.")
        return None, None

# Fonction pour charger les données de la NASA (simulée ici par un fichier local)
def load_data(lat, lon):
    # Simulation : Utilisation de données existantes
    df = pd.read_csv('nasa_power_data_with_date.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    return df

# Visualisation des tendances climatiques
def visualize_data(df):
    plt.figure(figsize=(12, 8))
    
    # Tracer la température moyenne, maximale et minimale
    plt.subplot(2, 1, 1)
    plt.plot(df['Date'], df['T2M'], label='Température (°C)', color='blue')
    plt.plot(df['Date'], df['TMAX'], label='Température Maximale (°C)', color='red')
    plt.plot(df['Date'], df['TMIN'], label='Température Minimale (°C)', color='green')
    plt.title('Températures (°C)')
    plt.legend()
    
    # Tracer les précipitations
    plt.subplot(2, 1, 2)
    plt.bar(df['Date'], df['PRECTOT'], label='Précipitations (mm)', color='orange', alpha=0.5)
    plt.title('Précipitations (mm)')
    plt.legend()
    
    st.pyplot(plt)

# Modélisation : Prévision des risques climatiques
def train_predictive_model(df):
    df = df.dropna()  # Supprimer les données manquantes
    
    # Définir les variables indépendantes (X) et la variable dépendante (y)
    X = df[['PRECTOT', 'RH2M', 'WS2M', 'ALLSKY_SFC_SW_DWN', 'TMAX', 'TMIN']]
    y = df['T2M']  # Prédire la température moyenne
    
    # Diviser les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Créer et entraîner un modèle de forêt aléatoire
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Prédire sur l'ensemble de test
    y_pred = model.predict(X_test)
    
    # Calculer les métriques d'évaluation du modèle
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    st.write(f"Mean Squared Error: {mse:.2f}")
    st.write(f"R2 Score: {r2:.2f}")
    
    return model

# Générer des recommandations basées sur les données actuelles
def generate_recommendations(df, model, threshold_temp=30):
    latest_data = df.iloc[-1]
    
    # Prédire la température pour les prochains jours en utilisant les dernières données
    X_latest = latest_data[['PRECTOT', 'RH2M', 'WS2M', 'ALLSKY_SFC_SW_DWN', 'TMAX', 'TMIN']].values.reshape(1, -1)
    predicted_temp = model.predict(X_latest)[0]
    
    st.write(f"\nTempérature Prédite: {predicted_temp:.2f}°C")
    
    if predicted_temp > threshold_temp:
        st.write("Alerte Sécheresse : Température élevée prévue, pensez à augmenter l'irrigation.")
    elif predicted_temp < 0:
        st.write("Alerte Gel : Risque de gel prévu, prenez des mesures pour protéger les cultures.")
    else:
        st.write("Conditions climatiques optimales pour les cultures.")

# Application principale Streamlit
def main():
    st.title("Outil de Prédiction Climatique pour les Agriculteurs")
    
    # Entrée de l'utilisateur : ville
    city_name = st.text_input("Entrez le nom de votre ville", "Paris")
    
    if st.button("Obtenir les données météorologiques"):
        lat, lon = get_lat_lon(city_name)
        
        if lat and lon:
            st.write(f"Coordonnées de {city_name} : Latitude = {lat}, Longitude = {lon}")
            
            # Charger les données
            df = load_data(lat, lon)
            
            # 1. Visualisation des tendances climatiques
            visualize_data(df)
            
            # 2. Entraîner le modèle prédictif
            model = train_predictive_model(df)
            
            # 3. Générer des recommandations pour les agriculteurs
            generate_recommendations(df, model, threshold_temp=30)

if __name__ == "__main__":
    main()


2024-10-05 16:14:49.982 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 16:14:49.983 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 16:14:49.984 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 16:14:49.985 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 16:14:49.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 16:14:49.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 16:14:49.987 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-05 16:14:49.988 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar